In [ ]:
import bct
import networkx
import scipy.io as sio
import seaborn as sns
import numpy as np 
import pandas as pd 
from statsmodels.stats.multitest import multipletests
import statsmodels.formula.api as smf
%matplotlib notebook
import pingouin

import plotly.express as px

from scipy import stats
from matplotlib import pyplot as plt

In [ ]:
atlaslabels = pd.read_csv('../../../utility/atlas_key.csv', header=None)

# Curvature Analysis

In [ ]:
for iqn in range(0, 2): 
    # Load Phenotype Data
    fn = '../../../../data/BehavioralAndDemographicData.xlsx'
    phenotype_df = pd.read_excel(fn)
    nsubjects = len(phenotype_df)

    connectomes_TP1 = np.zeros((83, 83, nsubjects))
    for n in range(0, connectomes_TP1.shape[2]): 
        fn = phenotype_df['ACTID'][n]
        fn = '../../../../data/curvature/' + fn + '_TP1.csv'
        a = np.loadtxt(fn, delimiter=',')
        a = np.array(a)
        connectomes_TP1[:, :, n] = a 

    connectomes_TP2 = np.zeros((83, 83, nsubjects))
    for n in range(0, connectomes_TP2.shape[2]): 
        fn = phenotype_df['ACTID'][n]
        fn = '../../../../data/curvature/' + fn + '_TP2.csv'
        a = np.loadtxt(fn, delimiter=',')
        a = np.array(a)
        connectomes_TP2[:, :, n] = a     
    
    if iqn == 0: 
        print('IQ less than 70')
    else: 
        print('IQ more than 70 ')

    inds = phenotype_df["IQ<70"].isin([iqn])
    connectomes_TP1 = connectomes_TP1[:, :, inds]
    connectomes_TP2 = connectomes_TP2[:, :, inds]
    phenotype_df = phenotype_df[phenotype_df["IQ<70"].isin([iqn])]
    phenotype_df = phenotype_df.reset_index()
    
    group_list = [["T2", "T3"], ["T1", "T3"], ["T1", "T2"]]
    for group_pair in group_list: 
        print(group_pair)

        group_phenotype_df = phenotype_df[phenotype_df['Group'].isin(group_pair)]
        list_group = phenotype_df['Group'].isin(group_pair).values
        group = group_phenotype_df['Group'].values
        age = group_phenotype_df['Age'].values
        nviq = group_phenotype_df['NVIQ'].values

        forumula_str = "tp2_scores ~ + tp1_scores + group + nviq + group:nviq + age"
        column_headers = ['tp2_scores', 'tp1_scores', 'group', 'age', 'nviq']
        region_pairs = [] 
        # Examine all edge pairs 
        pvalslist = [] 
        for n in range(0, 83): 
            for m in range(n, 83): 
                if m==n: 
                    continue 

                tp2_scores = connectomes_TP2[n, m, list_group]
                tp1_scores = connectomes_TP1[n, m, list_group]
                
                if len(np.unique(tp2_scores)) < 3: 
                    continue 
                if len(np.unique(tp1_scores)) < 3: 
                    continue
                
                if np.sum(tp2_scores) != 0: 
                    df = pd.DataFrame(columns=column_headers)
                    df['tp2_scores'] = tp2_scores
                    df['tp1_scores'] = tp1_scores
                    df['group'] = pd.Categorical(group)
                    df['nviq'] = nviq
                    df['age'] = age

                    glm = smf.glm(forumula_str, data=df).fit()
                    pval = glm.pvalues[1]
                    region_pairs.append([n, m])
                    pvalslist.append(pval)

        pvalslist = np.array(pvalslist)
        reject, pvals_corrected, a, b = multipletests(pvalslist, alpha=0.05, method='fdr_bh')
        print("Number of passing: ", sum(reject), "out of ", len(reject))

        rejected_edges = np.argwhere(reject==True)
        for ind in rejected_edges: 
            n = ind[0]
            r = region_pairs[n][0]
            c = region_pairs[n][1]
            print(atlaslabels[1][r][1:] + " - " + atlaslabels[1][c][1:], r, c)
            tp2_scores = connectomes_TP2[r, c, :]
            tp1_scores = connectomes_TP1[r, c, :]
            if "T3" in group: 
                print("Average change in allo streamlines: ", 
                      np.mean(tp2_scores[phenotype_df['Group']=="T1"] - tp1_scores[phenotype_df['Group']=="T1"]))
                print("Average change in placebo streamlines: ", 
                      np.mean(tp2_scores[phenotype_df['Group']=="T3"] - tp1_scores[phenotype_df['Group']=="T3"]))
                print("pvalue: ", pvalslist[n])
                print("qvalues: ", pvals_corrected[n])
            

In [ ]:
ind

In [ ]:
rejected_edges

In [ ]:
glm.pvalues

In [ ]:
iqn=1

# Load Phenotype Data
fn = '../../../../data/BehavioralAndDemographicData.xlsx'
phenotype_df = pd.read_excel(fn)
nsubjects = len(phenotype_df)

connectomes_TP1 = np.zeros((83, 83, nsubjects))
for n in range(0, connectomes_TP1.shape[2]): 
    fn = phenotype_df['ACTID'][n]
    fn = '../../../../data/curvature/' + fn + '_TP1.csv'
    a = np.loadtxt(fn, delimiter=',')
    a = np.array(a)
    connectomes_TP1[:, :, n] = a 

connectomes_TP2 = np.zeros((83, 83, nsubjects))
for n in range(0, connectomes_TP2.shape[2]): 
    fn = phenotype_df['ACTID'][n]
    fn = '../../../../data/curvature/' + fn + '_TP2.csv'
    a = np.loadtxt(fn, delimiter=',')
    a = np.array(a)
    connectomes_TP2[:, :, n] = a     

if iqn == 0: 
    print('IQ less than 70')
else: 
    print('IQ more than 70 ')

inds = phenotype_df["IQ<70"].isin([iqn])
connectomes_TP1 = connectomes_TP1[:, :, inds]
connectomes_TP2 = connectomes_TP2[:, :, inds]
phenotype_df = phenotype_df[phenotype_df["IQ<70"].isin([iqn])]
phenotype_df = phenotype_df.reset_index()

group_list = [["T2", "T3"], ["T1", "T3"], ["T1", "T2"]]
group_pair = group_list[1]

print(group_pair)

group_phenotype_df = phenotype_df[phenotype_df['Group'].isin(group_pair)]
list_group = phenotype_df['Group'].isin(group_pair).values
group = group_phenotype_df['Group'].values
age = group_phenotype_df['Age'].values
nviq = group_phenotype_df['NVIQ'].values

forumula_str = "tp2_scores ~ + tp1_scores + group + nviq + group:nviq + age"
column_headers = ['tp2_scores', 'tp1_scores', 'group', 'age', 'nviq']
region_pairs = [] 

# Examine all edge pairs 
pvalslist = [] 

n=12
m=45

tp2_scores = connectomes_TP2[n, m, list_group]
tp1_scores = connectomes_TP1[n, m, list_group]

if np.sum(tp2_scores) != 0: 
    df = pd.DataFrame(columns=column_headers)
    df['tp2_scores'] = tp2_scores
    df['tp1_scores'] = tp1_scores
    df['group'] = pd.Categorical(group)
    df['nviq'] = nviq
    df['age'] = age

    glm = smf.glm(forumula_str, data=df).fit()
    pval = glm.pvalues[1]
    region_pairs.append([n, m])
    pvalslist.append(pval)
    print(glm.summary())

pvalslist = np.array(pvalslist)
reject, pvals_corrected, a, b = multipletests(pvalslist, alpha=0.05, method='fdr_bh')
print("Number of passing: ", sum(reject), "out of ", len(reject))

rejected_edges = np.argwhere(reject==True)
for ind in rejected_edges: 
    n = ind[0]
    r = region_pairs[n][0]
    c = region_pairs[n][1]
    print(atlaslabels[1][r][1:] + " - " + atlaslabels[1][c][1:], r, c)
    tp2_scores = connectomes_TP2[r, c, :]
    tp1_scores = connectomes_TP1[r, c, :]
    if "T3" in group: 
        print("Average change in allo streamlines: ", 
              np.mean(tp2_scores[phenotype_df['Group']=="T1"] - tp1_scores[phenotype_df['Group']=="T1"]))
        print("Average change in placebo streamlines: ", 
              np.mean(tp2_scores[phenotype_df['Group']=="T3"] - tp1_scores[phenotype_df['Group']=="T3"]))
        print("pvalue: ", pvalslist[n])
            
            

In [ ]:
group_t1 = tp2_scores[phenotype_df['Group']=="T1"] - tp1_scores[phenotype_df['Group']=="T1"]
group_t3 = tp2_scores[phenotype_df['Group']=="T3"] - tp1_scores[phenotype_df['Group']=="T3"]
pingouin.compute_effsize(group_t1, group_t3)

# Treatment

In [ ]:
for iqn in range(0, 2): 
    # Load Phenotype Data
    fn = '../../../../data/BehavioralAndDemographicData.xlsx'
    phenotype_df = pd.read_excel(fn)
    nsubjects = len(phenotype_df)

    connectomes_TP1 = np.zeros((83, 83, nsubjects))
    for n in range(0, connectomes_TP1.shape[2]): 
        fn = phenotype_df['ACTID'][n]
        fn = '../../../../data/curvature/' + fn + '_TP1.csv'
        a = np.loadtxt(fn, delimiter=',')
        a = np.array(a)
        connectomes_TP1[:, :, n] = a 

    connectomes_TP2 = np.zeros((83, 83, nsubjects))
    for n in range(0, connectomes_TP2.shape[2]): 
        fn = phenotype_df['ACTID'][n]
        fn = '../../../../data/curvature/' + fn + '_TP2.csv'
        a = np.loadtxt(fn, delimiter=',')
        a = np.array(a)
        connectomes_TP2[:, :, n] = a     

    
    if iqn == 0: 
        print('IQ less than 70')
    else: 
        print('IQ more than 70 ')

    inds = phenotype_df["IQ<70"].isin([iqn])
    connectomes_TP1 = connectomes_TP1[:, :, inds]
    connectomes_TP2 = connectomes_TP2[:, :, inds]
    phenotype_df = phenotype_df[phenotype_df["IQ<70"].isin([iqn])]
    phenotype_df = phenotype_df.reset_index()
    

    group = phenotype_df['Treat'].values
    age = phenotype_df['Age'].values
    nviq = phenotype_df['NVIQ'].values

    forumula_str = "tp2_scores ~ + tp1_scores + group + nviq + group:nviq + age"
    column_headers = ['tp2_scores', 'tp1_scores', 'group', 'age', 'nviq']
    region_pairs = [] 
    # Examine all edge pairs 
    pvalslist = [] 
    for n in range(0, 83): 
        for m in range(n, 83): 
            if m==n: 
                continue 

            tp2_scores = connectomes_TP2[n, m, :]
            tp1_scores = connectomes_TP1[n, m, :]

            if len(np.unique(tp2_scores)) < 5: 
                    continue 
            if len(np.unique(tp1_scores)) < 5: 
                    continue
                
            
            if np.sum(tp2_scores) != 0: 
                df = pd.DataFrame(columns=column_headers)
                df['tp2_scores'] = tp2_scores
                df['tp1_scores'] = tp1_scores
                df['group'] = pd.Categorical(group)
                df['nviq'] = nviq
                df['age'] = age

                glm = smf.glm(forumula_str, data=df).fit()
                pval = glm.pvalues[1]
                region_pairs.append([n, m])
                pvalslist.append(pval)

    pvalslist = np.array(pvalslist)
    reject, pvals_corrected, a, b = multipletests(pvalslist, alpha=0.05, method='fdr_bh')
    print("Number of passing: ", sum(reject), "out of ", len(reject))

    rejected_edges = np.argwhere(reject==True)
    for ind in rejected_edges: 
        n = ind[0]
        r = region_pairs[n][0]
        c = region_pairs[n][1]
        print(atlaslabels[1][r][1:] + " - " + atlaslabels[1][c][1:], r, c)
        
        tp2_scores = connectomes_TP2[r, c, :]
        tp1_scores = connectomes_TP1[r, c, :]
        print("Average change in treatment streamlines: ", 
              np.mean(tp2_scores[phenotype_df['Treat']==1] - tp1_scores[phenotype_df['Treat']==1]))
        print("Average change in placebo streamlines: ", 
              np.mean(tp2_scores[phenotype_df['Treat']==0] - tp1_scores[phenotype_df['Treat']==0]))
        print("pvalue: ", pvalslist[n])
        print("qvalues: ", pvals_corrected[n])

In [ ]:
iqn=1

# Load Phenotype Data
fn = '../../../../data/BehavioralAndDemographicData.xlsx'
phenotype_df = pd.read_excel(fn)
nsubjects = len(phenotype_df)

connectomes_TP1 = np.zeros((83, 83, nsubjects))
for n in range(0, connectomes_TP1.shape[2]): 
    fn = phenotype_df['ACTID'][n]
    fn = '../../../../data/curvature/' + fn + '_TP1.csv'
    a = np.loadtxt(fn, delimiter=',')
    a = np.array(a)
    connectomes_TP1[:, :, n] = a 

connectomes_TP2 = np.zeros((83, 83, nsubjects))
for n in range(0, connectomes_TP2.shape[2]): 
    fn = phenotype_df['ACTID'][n]
    fn = '../../../../data/curvature/' + fn + '_TP2.csv'
    a = np.loadtxt(fn, delimiter=',')
    a = np.array(a)
    connectomes_TP2[:, :, n] = a     


if iqn == 0: 
    print('IQ less than 70')
else: 
    print('IQ more than 70 ')

inds = phenotype_df["IQ<70"].isin([iqn])
connectomes_TP1 = connectomes_TP1[:, :, inds]
connectomes_TP2 = connectomes_TP2[:, :, inds]
phenotype_df = phenotype_df[phenotype_df["IQ<70"].isin([iqn])]
phenotype_df = phenotype_df.reset_index()


group = phenotype_df['Treat'].values
age = phenotype_df['Age'].values
nviq = phenotype_df['NVIQ'].values

forumula_str = "tp2_scores ~ + tp1_scores + group + nviq + group:nviq + age"
column_headers = ['tp2_scores', 'tp1_scores', 'group', 'age', 'nviq']
region_pairs = [] 
# Examine all edge pairs 
pvalslist = [] 

n=61
m=65

tp2_scores = connectomes_TP2[n, m, :]
tp1_scores = connectomes_TP1[n, m, :]

if np.sum(tp2_scores) != 0: 
    df = pd.DataFrame(columns=column_headers)
    df['tp2_scores'] = tp2_scores
    df['tp1_scores'] = tp1_scores
    df['group'] = pd.Categorical(group)
    df['nviq'] = nviq
    df['age'] = age

    glm = smf.glm(forumula_str, data=df).fit()
    pval = glm.pvalues[1]
    region_pairs.append([n, m])
    pvalslist.append(pval)
    print(glm.summary())

pvalslist = np.array(pvalslist)
reject, pvals_corrected, a, b = multipletests(pvalslist, alpha=0.05, method='fdr_bh')
print("Number of passing: ", sum(reject), "out of ", len(reject))

rejected_edges = np.argwhere(reject==True)
for ind in rejected_edges: 
    n = ind[0]
    r = region_pairs[n][0]
    c = region_pairs[n][1]
    print(atlaslabels[1][r][1:] + " - " + atlaslabels[1][c][1:], r, c)

    tp2_scores = connectomes_TP2[r, c, :]
    tp1_scores = connectomes_TP1[r, c, :]
    print("Average change in treatment streamlines: ", 
          np.mean(tp2_scores[phenotype_df['Treat']==1] - tp1_scores[phenotype_df['Treat']==1]))
    print("Average change in placebo streamlines: ", 
          np.mean(tp2_scores[phenotype_df['Treat']==0] - tp1_scores[phenotype_df['Treat']==0]))
    print("pvalue: ", pvalslist[n])

In [ ]:
delta_scores = df['tp2_scores'].values - df['tp1_scores'].values
group = df['group'].values
delta_scores[group==0]
delta_scores[group==1]
pingouin.compute_effsize(delta_scores[group==1], delta_scores[group==0])